In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

#### Model Architecture

In [2]:
class AgeClassifier(nn.Module):
    def __init__(self):
        super(AgeClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(512 * 7 * 7, 1024)  # Adjust input size for fully connected layer
        self.fc2 = nn.Linear(1024, 1)  # Output a single value (age)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 512 * 7 * 7)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### Data Loading and Preprocessing

In [3]:
# Custom dataset to map folder names (ages) as integer labels
class AgeImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        image, label = super(AgeImageFolder, self).__getitem__(index)
        label = int(self.classes[label])  # Folder names are ages
        return image, label

# Define the transformations for the images, including data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomRotation(degrees=20),  # Rotate images by up to 20 degrees
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
])

# Load the dataset
dataset = AgeImageFolder(root='assessment-data', transform=transform)

# Split into train and test datasets (e.g., 80% train, 20% test)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders with no batching (batch_size=1)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

#### Training Setup: Loss Function and Optimizer

In [4]:
# Device configuration (GPU if available)
device = torch.device('mps')
# Initialize the model, loss function, and optimizer
model = AgeClassifier().to(device)
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
num_epochs = 100 # You can adjust this based on available time

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.float().to(device)  # Move data to the GPU if available
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))  # Ensure labels have the right shape
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Accumulate the running loss
        running_loss += loss.item()
    if epoch == 10:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

Epoch [11/100], Loss: 611.4441996040987


#### Evaluation

In [10]:
def evaluate(model, test_loader, threshold=5):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_samples = 0
    correct_predictions = 0  # To count the number of correct predictions
    
    criterion = nn.MSELoss()  # Loss function for evaluation
    
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.float().to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            
            total_loss += loss.item()
            total_samples += 1
            
            # Calculate number of correct predictions within the threshold
            predictions = outputs.squeeze().round()  # Round the predictions to the nearest integer
            correct_predictions += ((predictions - labels).abs() <= threshold).sum().item()  # Count correct predictions

    avg_loss = total_loss / total_samples  # Calculate average loss
    accuracy = correct_predictions / len(test_loader.dataset)  # Calculate accuracy as a fraction
    print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")  # Print average loss and accuracy
    return avg_loss, accuracy

In [11]:
test_loss = evaluate(model, test_loader)

Test Loss: 58.7774, Accuracy: 0.5714


In [12]:
# After the final training epoch
torch.save(model.state_dict(), 'age_classifier_model.pth')
print("Model saved successfully!")

Model saved successfully!


In [13]:
model.eval()

AgeClassifier(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=25088, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1, bias=True)
)

In [12]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Determine the device to use
device = torch.device("mps")

# Load the trained model
model = AgeClassifier().to(device)  # Ensure model is on the right device
model.load_state_dict(torch.load('age_classifier_model.pth', map_location=device))  # Load weights on the correct device
model.eval()  # Set the model to evaluation mode

# Define the transformations to match the training preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
])

# Start capturing from the webcam
cap = cv2.VideoCapture(0)  # 0 is usually the default camera

while True:
    ret, frame = cap.read()  # Capture a frame
    if not ret:
        break
    
    # Convert the frame from BGR to RGB
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    
    # Convert NumPy array (img) to PIL Image
    img = Image.fromarray(img)
    
    # Apply the transformations
    img = transform(img)  # Apply the transformations
    img = img.unsqueeze(0).to(device)  # Add batch dimension and move to the correct device

    # Make a prediction
    with torch.no_grad():
        output = model(img)
        predicted_age = output.item()  # Get the predicted age as a scalar

    # Display the predicted age on the frame
    cv2.putText(frame, f'Predicted Age: {int(predicted_age)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Show the live feed
    cv2.imshow('Live Age Prediction', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

NameError: name 'AgeClassifier' is not defined

#### Trnasfer learning with Data Augmentation

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import mean_absolute_error

In [7]:
# Data Augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset
train_dataset = datasets.ImageFolder(root='assessment-data', transform=transform_train)
test_dataset = datasets.ImageFolder(root='assessment-data', transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [8]:
model = models.resnet18(pretrained=True) # Pre-trained Model with Transfer Learning
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Predict a single continuous value (age)
device = model.to(de)
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Rate Scheduler

/Users/raman/Desktop/Assessment/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/raman/Desktop/Assessment/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
def train(model, train_loader, optimizer, criterion, scheduler, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            labels = labels.float().unsqueeze(1)  # Convert labels to float for regression
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

def test(model, test_loader):
    model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            labels = labels.float().unsqueeze(1)
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(outputs.cpu().numpy())

    mae = mean_absolute_error(true_labels, predicted_labels)
    print(f'Mean Absolute Error (Age Prediction): {mae:.2f} years')

# Training and Testing the Model
device = torch.device("mps")
model = model.to(device)

In [10]:
train(model, train_loader, optimizer, criterion, scheduler, num_epochs=100)
test(model, test_loader)

Epoch [1/100], Loss: 4.6212
Epoch [2/100], Loss: 4.4963
Epoch [3/100], Loss: 5.0263
Epoch [4/100], Loss: 4.4640
Epoch [5/100], Loss: 4.2594
Epoch [6/100], Loss: 4.4801
Epoch [7/100], Loss: 4.1463
Epoch [8/100], Loss: 4.1782
Epoch [9/100], Loss: 4.1028
Epoch [10/100], Loss: 4.0536
Epoch [11/100], Loss: 3.9949
Epoch [12/100], Loss: 4.1291
Epoch [13/100], Loss: 4.0073
Epoch [14/100], Loss: 4.1688
Epoch [15/100], Loss: 4.0461
Epoch [16/100], Loss: 4.0342
Epoch [17/100], Loss: 4.0743
Epoch [18/100], Loss: 3.9785
Epoch [19/100], Loss: 3.8699
Epoch [20/100], Loss: 3.9919
Epoch [21/100], Loss: 4.0095
Epoch [22/100], Loss: 4.0718
Epoch [23/100], Loss: 3.9744
Epoch [24/100], Loss: 3.9877
Epoch [25/100], Loss: 4.0005
Epoch [26/100], Loss: 3.9828
Epoch [27/100], Loss: 3.9955
Epoch [28/100], Loss: 3.9746
Epoch [29/100], Loss: 3.9290
Epoch [30/100], Loss: 3.9670
Epoch [31/100], Loss: 3.9778
Epoch [32/100], Loss: 3.9570
Epoch [33/100], Loss: 3.9750
Epoch [34/100], Loss: 4.0267
Epoch [35/100], Loss: 3

In [11]:
torch.save(model.state_dict(), "age_classifier.pth")

In [14]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Determine the device to use
device = torch.device("mps")

# Load the trained model
model = models.resnet18(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Custom layer for regression task
model.load_state_dict(torch.load('age_classifier.pth'))  # Replace with your model path
model.eval()

# Define the transformations to match the training preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
])

# Start capturing from the webcam
cap = cv2.VideoCapture(0)  # 0 is usually the default camera

while True:
    ret, frame = cap.read()  # Capture a frame
    if not ret:
        break
    
    # Convert the frame from BGR to RGB
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    
    # Convert NumPy array (img) to PIL Image
    img = Image.fromarray(img)
    
    # Apply the transformations
    img = transform(img)  # Apply the transformations
    img = img.unsqueeze(0).to(device)  # Add batch dimension and move to the correct device

    # Make a prediction
    with torch.no_grad():
        output = model(img)
        predicted_age = output.item()  # Get the predicted age as a scalar

    # Display the predicted age on the frame
    cv2.putText(frame, f'Predicted Age: {int(predicted_age)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Show the live feed
    cv2.imshow('Live Age Prediction', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

/var/folders/70/_xc8h2gj12b_8vjmwzk16r640000gn/T/ipykernel_6394/100731982.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('age_classifi

RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same